In [ ]:
pip install keras-tuner

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import tensorflow as tf
from keras_tuner import RandomSearch


df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:

x = df.drop(columns=['species'])
y = df['species']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test= le.transform(y_test)

In [ ]:
def test_model(hp):
    model=Sequential()
    model.add(Input(shape=(x_train.shape[1],)))
    model.add(Dense(hp.Int('unit1',min_value=32,max_value=128),activation=hp.Choice('activation1',['relu','tanh','sigmoid'])))
    model.add(Dense(hp.Int('unit2',min_value=32,max_value=128),activation=hp.Choice('activation2',['relu','tanh','sigmoid'])))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer=SGD(hp.Float('optimize',min_value=0.00001,max_value=0.001),momentum=hp.Float('momentum',min_value=0.0,max_value=0.9)),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model


In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    test_model,
    objective='val_accuracy',
    max_trials=5,  # You can adjust the number of trials
    directory='my_dir',  # Optional: Specify a directory to store results
    project_name='iris_tuning'  # Optional: Give your project a name
)
# Now use x_train,x_test instead of X_train,X_test and  x,y instead of X,y
# Ensure that y_train and y_test are Label Encoded before passing to tuner.search
x, y = df.drop(columns=['species']), df['species']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#The tuner is initialized with test_model function as a parameter
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
loss, accuracy = best_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy}")

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.4333333373069763

Best val_accuracy So Far: 0.4333333373069763
Total elapsed time: 00h 01m 41s
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.3323 - loss: 1.2824 - val_accuracy: 0.3333 - val_loss: 1.2475
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3583 - loss: 1.2378 - val_accuracy: 0.3333 - val_loss: 1.2367
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3490 - loss: 1.2269 - val_accuracy: 0.3333 - val_loss: 1.2265
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3396 - loss: 1.2408 - val_accuracy: 0.3333 - val_loss: 1.2170
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3187 - loss: 1.2452 - val_accuracy: 0.3333 - val_loss: 1.2082
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3479 - loss: 1.2273 - val_accuracy: 0.3333 - val_loss: 1.1998
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3021 - loss: 1.2725 - val_accuracy: 0.3333 - 